# Testing Accuracy Using Predicted Boxes from Phase 1


In [1]:
import cv2
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import random
import glob
import imutils
from collections import namedtuple
from utils import get_cropped_digits

In [9]:
new_templates = []
for i in range(2,92):
    tmp = cv2.imread(f'NEW_NUMBERS/{i}.jpg')
    gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    new_templates.append(thresh)
for row in range(1,11):
    for col in range(1,11):
        template = cv2.imread(f"Templates/numbers/PineTools.com_2023-05-08_07h18m28s/PineTools.com_files/row-{row}-column-{col}.png")
        gray = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        new_templates.append(thresh)
        
for row in [1,2,3,4,6]:
    for col in range(0,10):
        template = cv2.imread(f"Templates/numbers/templates/row-{row}-column-{col}.png")
        gray = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        new_templates.append(thresh)

In [3]:
win_size = (20, 20)   
block_size = (10, 10) 
block_stride = (5, 5) 
cell_size = (5, 5)    
nbins = 9             
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)

In [6]:
def test_accuracy(images_number):
    total_accuracy=0
    for i in range(1,images_number+1):
        img_accuracy=0
        digits,labels = get_cropped_digits(i)
        if len(digits) == 0:
            continue
        for j,digit in enumerate(digits):
            score=0
            label = labels[j]
            try:
                digit_resized = cv2.resize(digit, win_size)
            except:
                continue
            digit_hog = hog.compute(digit_resized).reshape(-1)
            scores = []
            for i,template in enumerate(new_templates):
                template_resized = cv2.resize(template, win_size)
                template_hog = hog.compute(template_resized).reshape(-1)
                score = np.dot(digit_hog, template_hog)
                scores.append(score)
            tmp = scores.index(max(scores))
            if tmp%10 == label:
                img_accuracy+=1
        img_accuracy/=len(digits)
        total_accuracy+=img_accuracy
        

    total_accuracy/=images_number
 

    print(f"Total Accuracy: {total_accuracy*100:0.3f}%")

In [8]:
test_accuracy(1000)

Total Accuracy: 43.517%
